Autor: Daniel Flores

# Credenciales

In [ ]:
import os
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GEN_TUTOR')
# Establecer la clave de API en las variables de entorno
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [ ]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec

from google.colab import userdata
PINECONE_API_KEY=userdata.get('Pinecone')

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

# Embedding Model


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddingsHF = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Lectura de Datos

## Lectura y Limpieza

### PDFS

In [1]:
!pip -q install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 12.1 MB/s eta 0:00:00


In [34]:
guia = '/content/drive/MyDrive/UTPL/CursoIntroMad/Guía didáctica_Curso Introducción.pdf'

In [6]:
plan = '/content/drive/MyDrive/UTPL/CursoIntroMad/Plan_docente.pdf'

### Lectura y Preprocesamiento Guía Didáctica

In [35]:
import fitz  # PyMuPDF
import re
import json

def clean_index(text):
    # Patrón para eliminar puntos suspensivos, números de página y líneas en blanco
    pattern = r'\.{10,}.*\n|^\d+\n|^\s*$\n'

    # Eliminar los elementos que coinciden con el patrón
    cleaned_text = re.sub(pattern, '', text, flags=re.MULTILINE)

    # Eliminar líneas en blanco adicionales
    cleaned_text = re.sub(r'\n\n', '\n', cleaned_text)

    return cleaned_text.strip()


# Leer PDF
def read_pdf_with_links(file_path):
    document = fitz.open(file_path)
    extracted_data = []
    current_context = None
    seen_links = set()

    for page_num in range(0, len(document)): #Indica desde que página empezará la lectura
        page = document.load_page(page_num)
        text = page.get_text()

        # Eliminar el número de página al final del texto
        text_lines = text.splitlines()
        if text_lines and text_lines[-1].strip().isdigit():
            text_lines = text_lines[:-1]  # Remover la última línea si es un número
        print
        # Convertir las líneas de vuelta a texto
        text = '\n'.join(text_lines)

        # Limpiar index si es una página de índice
        if any(keyword in text.lower() for keyword in ['Índice', '.......\n']):
            text = clean_index(text)

        # Extraer enlaces de la página con descripción enriquecida
        links = []
        for link in page.get_links():
            uri = link.get("uri", None)
            if uri and uri not in seen_links:
                seen_links.add(uri)

                # Intentar obtener una descripción más precisa del área del enlace
                rect = link.get("from")
                if rect:
                    description = page.get_textbox(rect).strip()
                else:
                    # Generar descripción contextual si no hay una explícita
                    description_context = text.split(uri)[0].splitlines()[-1] if uri in text else "No contextual description"
                    description = description_context.strip() or "Sin descripción disponible"
                links.append({"url": uri, "description": description})

        extracted_data.append({
            "page": page_num + 1,
            "text": text,
            "links": links
        })

    return extracted_data

In [63]:
# Ruta del archivo PDF
pdf_path = guia

# Leer el PDF y extraer texto con enlaces
pdf_data = read_pdf_with_links(pdf_path)

# Guardar los resultados en un archivo JSON
'''
output_path = "/content/guia_pdfv.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(pdf_data, f, ensure_ascii=False, indent=4)

print(f"Extracción completada. Datos guardados en {output_path}")
'''

'\noutput_path = "/content/guia_pdfv.json"\nwith open(output_path, "w", encoding="utf-8") as f:\n    json.dump(pdf_data, f, ensure_ascii=False, indent=4)\n\nprint(f"Extracción completada. Datos guardados en {output_path}")\n'

In [ ]:
type(pdf_data)

list

In [ ]:
len(pdf_data)

56

### Plan Docente

In [39]:
# Ruta del archivo PDF
pdf_path = plan

# Leer el PDF y extraer texto con enlaces
pdf_data = read_pdf_with_links(pdf_path)

### Generación de Chunks

In [8]:
!pip -q install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.0 MB/s eta 0:00:00


In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict
import re

# Función para crear chunks semánticos

def create_semantic_chunks(pdf_data: List[Dict], start_page: int = None) -> List[Dict]:
    """
    Create semantic chunks from PDF data using LangChain's RecursiveCharacterTextSplitter.
    Allows specifying a start page for chunking, keeping previous pages intact.

    Args:
        pdf_data (List[Dict]): List of dictionaries containing page number, text and links
        start_page (int, optional): Page number from which to start creating chunks

    Returns:
        List[Dict]: List of dictionaries containing either original pages or chunks
    """
    result = []

    # Separamos las páginas que se mantendrán intactas
    if start_page is not None:
        pages_to_keep = [page for page in pdf_data if page['page'] < start_page]
        pages_to_chunk = [page for page in pdf_data if page['page'] >= start_page]
    else:
        pages_to_keep = []
        pages_to_chunk = pdf_data

    # Mantenemos las páginas originales
    result.extend(pages_to_keep)

    # Si no hay páginas para hacer chunks, retornamos solo las páginas guardadas
    if not pages_to_chunk:
        return result

    # Procesamos cada página individualmente para evitar desajustes
    chunk_number = 1
    for page in pages_to_chunk:
        text = page['text']
        page_number = page['page']

        if 'links' in page and page['links']:
            for link in page['links']:
                if 'description' in link and 'url' in link:
                    text += f"\nLink relacionado: {link['description']} - {link['url']}\n"

        # Text Splitter
        text_splitter = RecursiveCharacterTextSplitter(
            separators=[
                "\n\n",           # Doble salto de línea primero (separa secciones principales)
                "\n(?=\d+\.)",    # Salto de línea seguido de número y punto (nuevas secciones numeradas)
                "\n(?=[A-Z])",    # Salto de línea seguido de mayúscula (nuevos temas)
                ". ",             # Punto y espacio (oraciones)
                " ",              # Espacio (palabras)
                ""               # Caracteres individuales
            ],
            chunk_size=1000,
            chunk_overlap=200,
            is_separator_regex=True
        )

        # Crear chunks para la página actual
        chunks = text_splitter.create_documents([text])

        for chunk in chunks:
            # Limpiamos el texto del chunk
            cleaned_text = re.sub(r'\s+', ' ', chunk.page_content).strip()

            # Solo incluimos chunks con contenido significativo
            if len(cleaned_text) > 50:
                chunk_dict = {
                    'page': page_number,
                    'chunk_number': chunk_number,
                    'text': cleaned_text
                }
                result.append(chunk_dict)
                chunk_number += 1

    return result

In [10]:
def analyze_chunks(chunks: List[Dict]) -> Dict:
    """
    Analiza los chunks creados y retorna estadísticas útiles.

    Args:
        chunks (List[Dict]): Lista de diccionarios con información de chunks

    Returns:
        Dict: Diccionario con estadísticas de los chunks
    """
    # Separamos páginas originales de chunks
    original_pages = [c for c in chunks if 'chunk_number' not in c]
    processed_chunks = [c for c in chunks if 'chunk_number' in c]

    chunk_lengths = [len(chunk['text']) for chunk in processed_chunks]

    return {
        "total_original_pages": len(original_pages),
        "total_chunks": len(processed_chunks),
        "avg_chunk_length": sum(chunk_lengths) / len(chunk_lengths) if chunk_lengths else 0,
        "min_chunk_length": min(chunk_lengths) if chunk_lengths else 0,
        "max_chunk_length": max(chunk_lengths) if chunk_lengths else 0,
    }

In [64]:
# Especificamos que queremos mantener las páginas 1-5 intactas y hacer chunks desde la página 6
chunks = create_semantic_chunks(pdf_data, start_page=6)

# Analizar los chunks creados
stats = analyze_chunks(chunks)
print(f"\nEstadísticas de procesamiento:")
print(f"Páginas originales mantenidas: {stats['total_original_pages']}")
print(f"Chunks creados: {stats['total_chunks']}")
print(f"Longitud promedio de chunks: {stats['avg_chunk_length']:.2f}")
print(f"Longitud mínima de chunks: {stats['min_chunk_length']}")
print(f"Longitud máxima de chunks: {stats['max_chunk_length']}")



Estadísticas de procesamiento:
Páginas originales mantenidas: 5
Chunks creados: 115
Longitud promedio de chunks: 553.27
Longitud mínima de chunks: 59
Longitud máxima de chunks: 991


In [ ]:
output_path = "/content/chunks_guia_intro.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=4)

print(f"Extracción completada. Datos guardados en {output_path}")


Extracción completada. Datos guardados en /content/chunks_guia_intro.json


In [ ]:
chunks

[{'page': 1,
  'text': 'Vicerrectorado de Modalidad Abierta y a Distancia\nIntroducción a la Modalidad a Distancia y en\nLínea \nGuía didáctica',
  'links': []},
 {'page': 2,
  'text': 'Universidad Técnica Particular de Loja \nGUÍA DEL ESTUDIANTE DE MODALIDAD A DISTANCIA\nGuía didáctica\nMaría José Rubio Gómez\nDiagramación y diseño digital\nEdiloja Cía. Ltda. \nTelefax: 593-7-2611418. \nSan Cayetano Alto s/n. \nwww.ediloja.com.ec \nedilojacialtda@ediloja.com.ec \nLoja-Ecuador\nAbril, 2021 \nDerecho de autor 016188\nPrimera edición \nISBN digital - 978-9942-39-173-5\nISBN físico - 978-9942-26-276-9\nLos  contenidos  de  este  trabajo  están  sujetos  a  una  licencia  internacional  Creative  Commons\nReconocimiento-NoComercial-CompartirIgual 4.0 (CC BY-NC-SA 4.0). Usted es libre de  Compartir —\ncopiar y redistribuir el material en cualquier medio o formato. Adaptar — remezclar, transformar y construir a\npartir del material citando la fuente, bajo los siguientes términos: Reconocimie

In [40]:
# Especificamos que queremos mantener las páginas 1-5 intactas y hacer chunks desde la página 6
chunks = create_semantic_chunks(pdf_data, start_page=1)

# Analizar los chunks creados
stats = analyze_chunks(chunks)
print(f"\nEstadísticas de procesamiento:")
print(f"Páginas originales mantenidas: {stats['total_original_pages']}")
print(f"Chunks creados: {stats['total_chunks']}")
print(f"Longitud promedio de chunks: {stats['avg_chunk_length']:.2f}")
print(f"Longitud mínima de chunks: {stats['min_chunk_length']}")
print(f"Longitud máxima de chunks: {stats['max_chunk_length']}")



Estadísticas de procesamiento:
Páginas originales mantenidas: 0
Chunks creados: 24
Longitud promedio de chunks: 600.42
Longitud mínima de chunks: 173
Longitud máxima de chunks: 969


In [12]:
chunks

[{'page': 1,
  'chunk_number': 1,
  'text': 'UNIVERSIDAD TÉCNICA PARTICULAR DE LOJA VICERRECTORADO DE MODALIDAD ABIERTA Y A DISTANCIA FACULTAD: Todas CARRERA: Todas PLAN DOCENTE DEL CURSO INTRODUCCIÓN A LA MODALIDAD A DISTANCIA Y EN LÍNEA (PROPEDÉUTICO) Diseñado por: Mariana Buele Maldonado Rita Cabrera González Mayra Eugenia Armijos MODALIDAD DE ESTUDIO: A DISTANCIA Y EN LÍNEA PERÍODO ACADÉMICO OCTUBRE 2024 - FEBRERO 2025'},
 {'page': 2,
  'chunk_number': 2,
  'text': '2 A. Datos de identificación del curso auto instruccional Curso: Introducción a la Modalidad a Distancia y en Línea (Propedéutico) Código: No aplica Tipo de asignatura (ECTS) No aplica Campo de formación del currículo (Rediseño) No aplica Número de créditos/horas: 58 horas Periodo académico ordinario/nivel: Antes del primer período académico Período académico ordinario/semestre: Octubre 2024 – febrero 2025 Nivel Grado Facultades: Facultad de Ciencias Económicas y Empresariales Facultad de Ingenierías y Arquitectura Facu

# Embeddings

## Vector Store

In [13]:
!pip -q install langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.3 MB/s eta 0:00:00


In [14]:
!pip -q install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.2 MB/s eta 0:00:00


In [15]:
!pip -q install langchain-huggingface

In [16]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddingsHF = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [22]:
import os
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GEN_TUTOR')
# Establecer la clave de API en las variables de entorno
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddingsGI = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [17]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec

from google.colab import userdata
PINECONE_API_KEY=userdata.get('Pinecone')

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [18]:
import time

index_name = "tutormad"  #cambiar de ser necesario

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

index = pc.Index(index_name)

In [61]:
from langchain_pinecone import PineconeVectorStore
# Crear la tienda de vectores en Pinecone
namespace = "curso_introduccion_2025"
vector_store = PineconeVectorStore(index=index, embedding=embeddingsHF, namespace=namespace) #puede reemplazarse el uso de HugginFace o el de pago

In [65]:
import json
from uuid import uuid4
from langchain.schema import Document
from langchain_pinecone import PineconeVectorStore


# Procesar los chunks para generar embeddings y preparar documentos
documents = []
ids = []

for chunk in chunks:
    chunk_id = str(uuid4())
    chunk_text = chunk["text"]
    metadata = {
        "page": chunk["page"],
        "chunk_number": chunk.get("chunk_number", 1)
    }

    document = Document(page_content=chunk_text, metadata=metadata)
    documents.append(document)
    ids.append(chunk_id)



# Insertar documentos con sus IDs únicos
vector_store.add_documents(documents=documents, ids=ids)

print(f"Se han insertado {len(documents)} documentos en el índice '{index_name}' bajo el namespace '{namespace}'.")


Se han insertado 120 documentos en el índice 'tutormad' bajo el namespace 'curso_introduccion_2025'.


# RAG


In [23]:
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",google_api_key=GOOGLE_API_KEY, temperature=0.3, max_tokens=500)

In [54]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def rag(pregunta):
  retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 6, "score_threshold": 0.5},
  )

  prompt_template = """
    Actúa como un asistente virtual académico especializado en brindar explicaciones claras y detalladas, con un tono entusiasta y amigable.
    Responde las siguientes preguntas basándote únicamente en el siguiente contexto,
    proporcionando ejemplos prácticos y explicaciones paso a paso cuando sea necesario,
    no menciones frases como 'según la información que tengo' o 'según los documentos que proporcionaste';
    si existe un link como recurso explicativo proporcionalo como recomendación;

    Contexto: {context}
    Pregunta: {question}
  """
  prompt = ChatPromptTemplate.from_template(prompt_template)

  chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
  )
  result = chain.invoke(pregunta)
  print(result)


Conteo de busquedas: 12


In [57]:
rag("cuáles son los requerimientos técnicos")

¡Hola! ¡Me emociona ayudarte con los requerimientos técnicos!  Para acceder y navegar correctamente por el EVA (Entorno Virtual de Aprendizaje), necesitas cumplir con ciertos requisitos.  Aunque no se detallan aquí, se indica que puedes encontrarlos en una infografía.  

Para acceder a esta información crucial, te recomiendo visitar este link: [https://view.genial.ly/6384d9af4620cc00139bce77](https://view.genial.ly/6384d9af4620cc00139bce77).  Revisar esta infografía antes de comenzar el curso te ayudará a evitar problemas de visualización y carga de recursos, ¡asegurando una experiencia de aprendizaje fluida y agradable!  ¡Anímate a explorarla!


In [55]:
rag("Qué significa la habilidad verbal? ¿Para qué sirve la habilidad verbal?")

¡Hola! ¡Me encanta ayudarte a entender la habilidad verbal!

La habilidad verbal es la capacidad de usar el lenguaje de forma efectiva, principalmente a través de la escritura.  Esto implica varias cosas: un buen manejo del vocabulario (conocer el significado de muchas palabras), comprender profundamente el significado de las palabras y frases, y la capacidad de conectar e identificar relaciones entre diferentes conceptos.  Piensa en ello como la destreza de usar las palabras como herramientas para comunicar ideas con precisión y claridad.

¿Para qué sirve? ¡Para muchísimas cosas!  La habilidad verbal es fundamental para el éxito académico y profesional.  Te permite:

* **Comprender textos:**  Ya sea un libro, un artículo científico, o un correo electrónico, la habilidad verbal te permite entender el mensaje del autor.  Esto es crucial para el aprendizaje y la investigación.
* **Expresar tus ideas:**  Es esencial para escribir ensayos, redactar informes, dar presentaciones, y comunicar

In [53]:
rag("Qué significa la habilidad verbal? ¿Para qué sirve la habilidad verbal?")

¡Hola! ¡Me encanta ayudarte a entender la habilidad verbal!

La habilidad verbal es, básicamente, tu capacidad para entender y usar el lenguaje de forma efectiva.  Se trata de mucho más que solo saber leer y escribir; implica comprender el significado de las palabras,  relacionar conceptos entre sí, y expresar tus ideas de manera clara y precisa, tanto de forma escrita como oral.

Piensa en esto:  ¿Puedes entender el significado de una palabra compleja como "eufemismo"? ¿Puedes identificar la diferencia entre sinónimos como "grande" y "enorme"? ¿Puedes resumir un texto largo y complejo en tus propias palabras?  Si respondiste sí a estas preguntas, ¡ya estás demostrando tu habilidad verbal!

¿Para qué sirve? ¡Para muchísimas cosas!  En la universidad, te ayudará a:

* **Comprender tus lecturas:**  Entender textos académicos, artículos científicos, y libros de texto es crucial para el éxito universitario. La habilidad verbal te permitirá extraer la información clave y analizarla críticam

In [52]:
rag("¿Qué significa la habilidad verbal? ¿Para qué sirve la habilidad verbal? ")

¡Hola! ¡Me encanta ayudarte a entender la habilidad verbal!  Es una habilidad fascinante y muy importante.

La habilidad verbal se refiere a tu capacidad de usar y comprender el lenguaje de manera efectiva, tanto en forma escrita como oral.  Esto incluye varias cosas:

* **Manejo del vocabulario:**  Saber el significado de muchas palabras y usarlas apropiadamente.  Por ejemplo, saber la diferencia entre "indiferente" y "apático", o usar "sinónimo" en lugar de "palabra que significa lo mismo".

* **Comprensión de significados:**  Entender el significado de oraciones, párrafos y textos completos.  No solo leer las palabras, sino captar la idea principal y los detalles importantes.  Imagina leer un artículo de noticias: la habilidad verbal te permite entender no solo lo que pasó, sino también el contexto y las implicaciones.

* **Relación de conceptos:**  Conectar ideas y conceptos entre sí.  Por ejemplo, entender cómo se relaciona la causa y el efecto en un texto, o identificar la analog

In [31]:
rag("en que semana Participe en el curso de competencias informacionales de Biblioteca.")

¡Hola!  La información proporcionada no especifica la semana exacta en la que se debe participar en el curso de competencias informacionales de la Biblioteca.  Solo indica que es una actividad que debe completarse.


In [50]:
rag("Cómo edito mi perfil")

¡Genial que quieras editar tu perfil!  Es muy fácil.  En la sección "Cuenta", encontrarás la opción "Editar perfil".  Allí podrás personalizar tu Entorno Virtual de Aprendizaje (EVA) ingresando tu información personal y añadiendo una foto de perfil para que tus compañeros te conozcan y puedan contactarte.  Recuerda revisar toda la información antes de guardar los cambios.  Para ayudarte aún más, te recomiendo este vídeo tutorial: https://www.youtube.com/embed/JyXUadwgfd0?rel=0


In [48]:
#Con plan
rag("Cuando es el primer examen presencial?")

¡Hola!  No se especifica la fecha exacta del primer examen presencial en los documentos proporcionados.  Sin embargo, sí se menciona que habrá un encuentro virtual al finalizar la semana 5 del curso para resolver dudas.  Este encuentro se anunciará con antelación a través de los canales de comunicación oficiales de la universidad. ¡Mantente atento a tus correos y a las plataformas de la universidad para no perderte la información!


In [ ]:
#Antes de añadir plan docente
rag("Qué actividades tenemos para la semana 1?")

¡Hola!  No puedo darte información específica sobre las actividades de la semana 1 porque el texto proporcionado no detalla las actividades por semana.  Sin embargo, sí puedo decirte cómo encontrar esa información.

El texto menciona un **Calendario** dentro del curso.  Este calendario te mostrará las tareas y actividades planificadas, incluyendo las fechas límite de entrega.  ¡Es una herramienta genial para organizar tu tiempo y no perderte ninguna fecha importante!  Revisar el calendario es la mejor manera de saber qué actividades tienes para la semana 1 (o cualquier otra semana).

Además, el texto menciona una **bandeja de entrada** donde puedes recibir mensajes de tus profesores y compañeros.  Es posible que recibas recordatorios o información adicional sobre las actividades de la semana 1 a través de este medio.  ¡Mantén un ojo en tu bandeja de entrada para estar al día!



In [ ]:
rag("""Ayudame a responder estas preguntas:  1. En la bandeja de entrada se puede enviar mensajes a:
a. Cualquier usuario con cuenta UTPL.
b. Docentes y compañeros de clase.
c. A una cuenta de correo electrónico alternativa.
2. Herramienta que ayuda a recordar las actividades.
a.
b.
c.
3.
a.
b.
c.
Notificaciones.
 Wiki.
 Calendario.
 En la interacción de mensajes, los enviados o recibidos provenientes de
 la misma persona, se presentan mediante:
Un punto verde en la parte superior derecha.
 Un número en la parte superior izquierda.
 Un número en la parte superior derecha.""")

¡Hola! ¡Con gusto te ayudo a responder estas preguntas!  Vamos paso a paso:

**Pregunta 1: En la bandeja de entrada se puede enviar mensajes a:**

La respuesta correcta es **b. Docentes y compañeros de clase.**  El texto indica explícitamente que sí se pueden enviar mensajes a docentes y compañeros de clase.  Las otras opciones no están confirmadas en la información proporcionada.

**Pregunta 2: Herramienta que ayuda a recordar las actividades:**

La respuesta correcta es **c. Calendario.** El texto menciona que el calendario sirve al estudiante para recordar actividades.

**Pregunta 3: En la interacción de mensajes, los enviados o recibidos provenientes de la misma persona, se presentan mediante:**

La respuesta correcta es **c. Un número en la parte superior derecha.** El texto indica que el EVA (Entorno Virtual de Aprendizaje) muestra la interacción de mensajes con un número en la parte superior derecha.


¡Espero que esto te haya sido útil!  Si tienes más preguntas, ¡no dudes en pr

In [ ]:
rag("Cómo envío mensajes en el eva?")

¡Hola! ¡Enviando mensajes en el EVA es super fácil!  Puedes hacerlo a través de la **bandeja de entrada**.  Piensa en ella como tu buzón de correo dentro del EVA.

Aquí te explico cómo funciona:

1. **Acceso a la Bandeja de Entrada:**  La bandeja de entrada te permite enviar y recibir mensajes a tus profesores y compañeros.  Es genial para la colaboración!

2. **Enviar Mensajes:**  Dentro de la bandeja de entrada, encontrarás la opción para redactar un nuevo mensaje.  Allí podrás escribir tu mensaje y selecciona a quién se lo quieres enviar (docentes o compañeros).

3. **Notificaciones:** ¡Genial! Recibirás notificaciones de nuevos mensajes en la cuenta de correo electrónico que registraste en el EVA.  Así que, ¡no te perderás ningún mensaje importante!  Recuerda agregar un correo electrónico alternativo que uses con frecuencia para estar siempre al día.  Puedes configurar esto en la sección de "Editar perfil".  ¡Te recomiendo revisar la configuración de notificaciones para personaliza

In [ ]:
rag("como me comunico con soporte?")

¡Genial que quieras comunicarte con soporte!  Tienes dos opciones principales para obtener ayuda con el EVA:

1. **Chat en vivo:** Esta es una manera rápida y directa de hablar con un representante de soporte técnico.  Puedes acceder al chat en vivo a través de este enlace: [https://view.genial.ly/61a8d4a1e936b00d944097dc](https://view.genial.ly/61a8d4a1e936b00d944097dc).  ¡Simplemente haz clic en el enlace y comienza a chatear!

2. **Buzón de consultas:** Si prefieres una comunicación escrita o si tu consulta requiere más tiempo para ser respondida, puedes usar el buzón de consultas de la UTPL.  Aunque el texto no detalla cómo acceder a este buzón, es una opción adicional para contactar a la universidad para cualquier duda que tengas.


¡Espero que esto te ayude!  No dudes en usar cualquiera de estas opciones para obtener la asistencia que necesitas.



In [ ]:
rag("Cómo edito mi perfil")

¡Hola! ¡Editar tu perfil es facilísimo!  Hay varias maneras de hacerlo, ¡y te las explicaré todas!

Primero, puedes ir a la sección "Cuenta".  Allí encontrarás la opción "Editar perfil", donde podrás personalizar tu Entorno Virtual de Aprendizaje (EVA).  Podrás ingresar tu información personal y subir una foto de perfil para que tus compañeros te conozcan. ¡Recuerda revisar toda la información antes de guardar los cambios!

Además, en la sección "Perfil", también puedes actualizar tu foto y biografía.  Es como darle un toque personal a tu espacio en el EVA.

Para ayudarte aún más, te recomiendo ver este video tutorial: [https://www.youtube.com/embed/JyXUadwgfd0?rel=0](https://www.youtube.com/embed/JyXUadwgfd0?rel=0)


¡Anímate a personalizar tu perfil!  Es una excelente manera de conectar con tus compañeros y docentes.



In [ ]:
rag("Qué es el razonamiento verbal?")

¡Hola! ¡Qué genial que quieras aprender sobre razonamiento verbal!  Es un tema fascinante.  Según la información que tengo, el razonamiento verbal se refiere a un conjunto de actividades mentales que consisten en conectar ideas siguiendo ciertas reglas.  Piensa en ello como un rompecabezas mental donde tienes que unir las piezas (ideas) para llegar a una conclusión.

Miller Charles (2007) lo define como "el conjunto de actividades mentales que consisten en la conexión de ideas de acuerdo con ciertas reglas".  Es decir, no es solo pensar, sino pensar de manera organizada y lógica.

Para entenderlo mejor, el razonamiento verbal implica trabajar con diferentes tipos de relaciones entre palabras, como:

* **Sinónimos:** Palabras con significados similares (ej: grande/enorme, feliz/alegre).
* **Antónimos:** Palabras con significados opuestos (ej: caliente/frío, bueno/malo).
* **Analogías:** Relaciones de semejanza entre dos pares de palabras (ej: perro:ladrar :: gato:maullar -  un perro lad

In [ ]:
rag("link de la autoevaluación 1")

¡Genial que quieras empezar con las autoevaluaciones!  El link para la Autoevaluación 1 es: https://view.genial.ly/60f3406a8839f20d8cdb3578

¡Mucha suerte y a por todas!  Recuerda que estas autoevaluaciones te ayudarán a identificar tus fortalezas y áreas de mejora. ¡Empezarás a ver resultados increíbles!



# Preguntas al RAG

## Sin límite de tokens y con Flash 1.5:

In [ ]:
rag("como uso el EVA?")

El texto describe el EVA como un espacio de interacción para estudiantes, accesible desde una aplicación móvil (Canvas Student) y a través de un navegador web.  Su uso abarca diversas funciones:

**1. Comunicación:**

* **Bandeja de entrada:** Permite mantenerse comunicado desde cualquier lugar.

**2. Aprendizaje:**

* **Autoevaluaciones:**  Se ofrecen autoevaluaciones para verificar el conocimiento sobre temas específicos (ej: verdadero/falso sobre el tema 1.1).  Estas proporcionan retroalimentación para reforzar el aprendizaje.
* **Actividades del curso:** El EVA contiene diferentes tipos de actividades para verificar el logro de los resultados de aprendizaje, incluyendo:
    * **Tareas:**  Envío de trabajos.
    * **Evaluaciones:** Pruebas o exámenes.
    * **Foros de discusión:** Participación en debates.
    * **Chat y videocolaboración:**  Interacción en tiempo real con otros estudiantes o profesores.  Cada actividad tiene parámetros como calificación, fechas límite, duración e i

## Sin limite de tokens con Flash 8B

In [ ]:
#Sin limite de tokens pero con el flash 8B
rag("Como uso el EVA?")

El EVA (Entorno Virtual de Aprendizaje) te permite interactuar y acceder a recursos de aprendizaje.  Para utilizarlo correctamente, debes seguir estos pasos y considerar las diferentes herramientas:

**1. Navegación:**

* **Menú de navegación global:**  Te permite acceder a diferentes secciones del curso, como contenido, herramientas y navegación del curso.  Observa la imagen 1, 2, 3 y 4 (si las imágenes están disponibles) para identificar las opciones en el menú.
* **Bandeja de entrada:**  Mantente comunicado con la plataforma desde cualquier lugar.  Utiliza esta función para interactuar con tus compañeros y tutores.

**2. Actividades de aprendizaje:**

* **Autoevaluaciones:**  El EVA ofrece autoevaluaciones para que evalúes tus conocimientos.  Un ejemplo es la autoevaluación sobre los temas 1 al 4, que se basa en preguntas de verdadero o falso sobre el tema 1.1.  Cada respuesta te proporciona retroalimentación.
* **Tareas, evaluaciones y foros:**  Estas son actividades comunes en el 

## Con límite de tokens (500) y flash 1.5:

In [ ]:
rag("como uso el EVA?")

El texto describe el EVA como un espacio de interacción en línea, accesible también a través de una aplicación móvil llamada Canvas Student.  Su uso abarca varias funciones clave:

**1. Comunicación:**  La bandeja de entrada del EVA permite mantenerse comunicado desde cualquier lugar.

**2. Aprendizaje:** El EVA contiene:

* **Contenidos del curso:**  Material de aprendizaje organizado en unidades y temas.
* **Actividades de aprendizaje:**  Para verificar el logro de los resultados de aprendizaje.  Estas incluyen:
    * **Tareas:**  Envío de trabajos.
    * **Evaluaciones:**  Pruebas o exámenes.
    * **Foros de discusión:**  Espacios para interactuar con otros estudiantes y el profesor.
    * **Chat y videocolaboración:**  Herramientas para comunicación en tiempo real.  Cada actividad tiene parámetros como calificación, fechas límite, duración e intentos permitidos. Es crucial prestar atención a esta parametrización.
* **Autoevaluaciones:**  Para comprobar el conocimiento adquirido so

In [ ]:
rag("Dime 3 preguntas para comprobar mi entendimiento del EVA")

Basándome en los documentos proporcionados, aquí hay 3 preguntas para comprobar tu entendimiento del EVA (Entorno Virtual de Aprendizaje), junto con las respuestas esperadas y explicaciones:


**Pregunta 1:** ¿De qué maneras puedo ordenar la lista de calificaciones en el EVA?

**Respuesta esperada:**  La lista de calificaciones se puede ordenar por fecha de entrega, grupo de tareas, módulo y título de la actividad. La fecha de entrega es la opción predeterminada.

**Explicación:**  El documento menciona explícitamente estas opciones de ordenamiento en la sección de calificaciones.  Esto permite al estudiante una mayor flexibilidad para revisar su progreso en el curso.


**Pregunta 2:** ¿Qué tipos de actividades de aprendizaje puedo encontrar en el EVA, y qué información importante debo considerar al participar en ellas?

**Respuesta esperada:** El EVA incluye tareas, evaluaciones, foros de discusión, chat y videocolaboración.  Al participar, es crucial considerar la parametrización de 

## Con límite de tokens (500) y flash 8B:

In [ ]:
rag("como uso el EVA?")

El documento describe cómo usar el Entorno Virtual de Aprendizaje (EVA) de la UTPL.  No hay una única respuesta a "cómo usar el EVA", ya que depende de la actividad específica que se quiera realizar.  Sin embargo, el documento proporciona información sobre varias funcionalidades y recursos.

Aquí te presento una guía paso a paso sobre cómo usar algunas de las funcionalidades más comunes:

**1. Navegación y Comunicación:**

* **Bandeja de entrada:** El EVA te permite mantenerte comunicado desde cualquier lugar.  Para usarla, debes acceder a la bandeja de entrada del EVA.  No se detalla cómo acceder a ella, pero se asume que es parte de la interfaz principal.
* **Menú de navegación global, menú de herramientas del curso, menú de navegación del curso:**  Estos menús te guían por el contenido del curso, las herramientas disponibles y la estructura del EVA.  Para usarlos, navega por las opciones disponibles en la interfaz principal.

**2. Actividades de Aprendizaje:**

* **Autoevaluaciones:

In [ ]:
rag("Dime 3 preguntas para comprobar mi entendimiento del EVA")

Basándome en el texto proporcionado, aquí tienes 3 preguntas para comprobar tu entendimiento del Entorno Virtual de Aprendizaje (EVA):

1. **¿Qué opciones de ordenamiento ofrece la sección de "Calificaciones" del EVA, y cuál es la predeterminada?**  (Esta pregunta evalúa tu comprensión de la funcionalidad de la sección de calificaciones dentro del EVA, incluyendo la capacidad de ordenar por diferentes criterios y el conocimiento del criterio predeterminado).


2. **Describe tres tipos de actividades de aprendizaje que se pueden encontrar en un EVA, y explica brevemente cómo se diferencian entre sí.** (Esta pregunta evalúa tu comprensión de la variedad de actividades que un EVA puede ofrecer, incluyendo tareas, evaluaciones y foros de discusión, y tu capacidad para distinguirlas).


3. **¿Cómo se pueden gestionar las notificaciones en el EVA y qué información adicional se puede registrar en la sección de "Canales de contacto"?** (Esta pregunta evalúa tu comprensión de la configuración p

## Otras Preguntas

In [ ]:
rag("que debo hacer en la semana 2")

Basándome únicamente en los documentos proporcionados, no hay información específica sobre qué se debe hacer en la semana 2.  Los documentos describen aspectos del entorno virtual de aprendizaje (EVA) de la UTPL, incluyendo el uso del calendario, la bandeja de entrada, la gestión de archivos y la autoevaluación de habilidades (mencionada para la semana 4).  No hay un plan de estudios o calendario detallado que indique las tareas para la semana 2.



In [ ]:
rag("que debo realizar en la semana 4")

La información proporcionada no especifica qué actividades se deben realizar en la semana 4.  Los documentos describen las funcionalidades de la plataforma UTPL (Calendario, Bandeja de entrada, Archivos, Perfil, etc.), pero no incluyen un plan de estudios o un calendario con las tareas semanales.  Para saber qué hacer en la semana 4, necesitaría consultar el programa del curso o el calendario académico proporcionado por su instructor.



In [ ]:
rag("qué permite la herramienta studio?")

Basado en el texto proporcionado, la herramienta Studio permite grabar, cargar y compartir videos.  También permite agregar videos desde un computador y desde YouTube.



In [ ]:
rag("En que semana es la Autoevaluación de habilidades básicas y compromiso de mejora para el estudio en la carrera?")

La autoevaluación de habilidades básicas y compromiso de mejora para el estudio en la carrera se realiza en la cuarta semana.



In [ ]:
rag("de que trata la guía didáctica?")

Basándome únicamente en los documentos proporcionados, la guía didáctica del curso "Introducción a la Modalidad a Distancia y en Línea" trata sobre la familiarización con el Entorno Virtual de Aprendizaje (EVA) y el sistema de educación a distancia de la universidad.  

Específicamente, se enfoca en:

* **Familiarización con el EVA (Unidad 1):**  El documento menciona un video introductorio e indicaciones generales para navegar y utilizar el EVA.  El objetivo es que el estudiante se familiarice con las funcionalidades del EVA para desarrollar las actividades del curso.  Se evalúa mediante un cuestionario de 10 puntos.

* **Sistema de Educación a Distancia (Unidad 2):**  Esta unidad cubre el modelo educativo y pedagógico de la modalidad a distancia y en línea, además de adquirir competencias relacionadas con hábitos y técnicas de estudio.  También se evalúa con un cuestionario de 10 puntos.

* **Fase Presencial:**  La guía menciona talleres presenciales en centros de apoyo al final del 

In [ ]:
rag("")